In [1]:
import os
import sys
import math
import random
import joblib
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
test = pd.read_excel(os.path.join('..', '..', '0.Data', '02_평가데이터', '2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx'), skiprows = 1)
test_meta = pd.read_excel(os.path.join('..', '..', '0.Data', '02_평가데이터', 'test수작업_meta.xlsx'))

In [6]:
test_meta.shape

(417, 12)

In [8]:
test_meta = test_meta.merge(test[['상품코드', '판매단가']].drop_duplicates(), on = '상품코드', how = 'left')

In [77]:
test_meta.to_excel(os.path.join('..', '..', '0.Data', '02_평가데이터', 'test수작업_meta.xlsx'), index = False)

In [11]:
item = pd.read_excel('./item.xlsx')

In [12]:
item

,NEW상품코드,NEW상품명,상품군,NEW_최고판매단가,NEW_최저판매단가,NEW_평균판매단가,NEW_중간판매단가,NEW_최고-최저,NEW_분산,NEW_표준편차,전체_가격대
0,0,가이거 남성 블랙 에디션 다이아몬드 워치,잡화,178000.0,178000.0,178000.0,178000.0,0.0,0.0,0.000000,고가
1,1,가이거 무빙스타 다이아몬드 워치,잡화,129000.0,119000.0,124000.0,124000.0,10000.0,50000000.0,7071.067812,고가
2,2,가이거 무빙스타 다이아몬드 워치 + 보석함,잡화,119000.0,99000.0,109000.0,109000.0,20000.0,200000000.0,14142.135624,고가
3,3,가이거 여성 블랙 에디션 다이아몬드 워치,잡화,178000.0,178000.0,178000.0,178000.0,0.0,0.0,0.000000,고가
4,4,가이거 제니스시계 주얼리세트,잡화,139000.0,129000.0,134000.0,134000.0,10000.0,50000000.0,7071.067812,고가
...,...,...,...,...,...,...,...,...,...,...,...
1618,1618,QISS 프리미어 아침에 끌리다 마시는한끼 21통,농수축,56900.0,56900.0,56900.0,56900.0,0.0,0.0,0.000000,저가
1619,1619,QISS 프리미어 아침에 끌리다 마시는한끼 42통,농수축,76900.0,76900.0,76900.0,76900.0,0.0,0.0,0.000000,중저가
1620,1620,W클라우드 남성 데일리 트레킹화 3종,잡화,49800.0,49800.0,49800.0,49800.0,0.0,0.0,0.000000,저가
1621,1621,W클라우드 여성 데일리 트레킹화 3종,잡화,49800.0,49800.0,49800.0,49800.0,0.0,0.0,0.000000,저가


In [169]:
data = joblib.load(os.path.join('..', '..', '0.Data', '05_분석데이터', 'test_FE.pkl'))

In [165]:
data = joblib.load(os.path.join('..', '..', '0.Data', '05_분석데이터', 'train_FE.pkl'))

In [170]:
locals().update(data)

In [171]:
X.to_excel('./test.xlsx', index = False)

In [167]:
X.shape

(35379, 401)